# Stable Audio
https://huggingface.co/stabilityai/stable-audio-open-1.0




In [4]:
!pip install torchsde
import torch
import soundfile as sf
import zipfile # for zipfile features
from diffusers import StableAudioPipeline
from IPython.display import Audio  # Import the Audio module for playback

pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")


# define the prompts
sound_requests = [
    {
        "prompt": "Sharp parrot scream layered with storm wind and distant animal calls, wild and dramatic.",
        "duration": 6.0,
        "filename": "sound_1.wav"
    },
    {
        "prompt": "Fast deer breathing and rapid hoof movement through dense jungle brush, urgent and cinematic tension.",
        "duration": 10.0,
        "filename": "sound_2.wav"
    },
    {
        "prompt": "Intense rapid crab claw clicks on wet stone mixed with sudden fish splashes in shallow water, rising echo.",
        "duration": 8.0,
        "filename": "sound_3.wav"
    },
    {
        "prompt": "Piercing eagle cry fading into deep owl hoots and distant wolf howl, dark jungle night, wide atmospheric reverb, while continue with whole duration",
        "duration": 15.0,
        "filename": "sound_4.wav"
    },
]
negative_prompt = "Low quality."

# set the seed for generator
generator = torch.Generator("cuda")

# run the generation
generated_files = []

for request in sound_requests:
    audio = pipe(
        request["prompt"],
        negative_prompt=negative_prompt,
        num_inference_steps=200,
        audio_end_in_s=request["duration"],
        num_waveforms_per_prompt=1,
        generator=generator,
    ).audios

    output = audio[0].T.float().cpu().numpy()
    sf.write(request["filename"], output, pipe.vae.sampling_rate)
    generated_files.append(request["filename"])

# Create ZIP file
zip_filename = "sounds.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in generated_files:
        zipf.write(file)

print(f"ZIP file created: {zip_filename}")

# Play the first audio
Audio(generated_files[0])

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

ZIP file created: sounds.zip
